In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Input, Concatenate, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model

In [ ]:
data = pd.read_csv('new_df.csv')
data = pd.DataFrame(data)

<ipython-input-3-d3c71521113a>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('new_df.csv')


In [ ]:
lag = 3
for i in range(1, lag + 1):
  data[f'pm10-{i}'] = data['pm10'].shift(i)

In [ ]:
data = data.dropna()

In [ ]:
data = data.sort_values(by='Stamp')
data.reset_index(drop=True, inplace=True)

In [ ]:
data

,SensorId,Stamp,humidity,pm10,pm25,temperature,Hour,Month_Day_Calculation,pm10-1,pm10-2,pm10-3
0,11888f3a-bc5e-4a0c-9f27-702984decedf,2023-01-01 00:00:00+00:00,99.0,159.0,77.0,5.0,0.00,1.032258,14.0,14.0,14.0
1,1587b09d-d0b0-4f46-b05e-d2fb93fc7863,2023-01-01 00:00:00+00:00,74.0,90.0,50.0,7.0,0.00,1.032258,18.0,18.0,24.0
2,b80e5cd2-76cb-40bf-b784-2a0a312e6264,2023-01-01 00:00:00+00:00,91.0,112.0,76.0,6.0,0.00,1.032258,29.0,37.0,10.0
3,8defa36a-62ca-448a-9ffb-5a2848c2dfa0,2023-01-01 00:00:00+00:00,81.0,127.0,58.0,5.0,0.00,1.032258,7.0,3.0,8.0
4,cec29ba1-5414-4cf3-bbcc-8ce4db1da5d0,2023-01-01 00:00:00+00:00,66.0,97.0,60.0,8.0,0.00,1.032258,3.0,4.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...
2263910,60aa65c6-b7c4-4aae-8769-5dbcba356b5d,2024-11-21 22:45:00+00:00,73.0,35.0,19.0,2.0,22.75,11.700000,43.0,46.0,43.0
2263911,661b61de-e128-4966-ad32-22924713d1d5,2024-11-21 22:45:00+00:00,35.0,30.0,18.0,15.0,22.75,11.700000,32.0,33.0,34.0
2263912,2c275060-799f-46c9-a2eb-fc613c4b4ff6,2024-11-21 22:45:00+00:00,78.0,49.0,27.0,0.0,22.75,11.700000,36.0,41.0,56.0
2263913,cf22ad10-aa16-4275-9d05-1830c16ec312,2024-11-21 22:45:00+00:00,63.0,32.0,20.0,4.0,22.75,11.700000,35.0,36.0,28.0


In [ ]:
features = ['humidity', 'temperature', 'pm10-1', 'pm10-2', 'pm10-3', 'Hour', 'Month_Day_Calculation']
target = 'pm10'

In [ ]:
X = data[features].values
y = data[target].values

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, shuffle=False
)

In [ ]:
time_series_input = Input(shape=(1, X_scaled.shape[2]))

In [ ]:
x = LSTM(64, return_sequences=False)(time_series_input)
x = Dropout(0.2)(x)

In [ ]:
x = Dense(32, activation='relu')(x)
output = Dense(1)(x)

In [ ]:
model = Model(inputs=[time_series_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=1,
    batch_size=32,
    validation_data=(X_test, y_test)
)

56598/56598 ━━━━━━━━━━━━━━━━━━━━ 189s 3ms/step - loss: 342.0277 - val_loss: 735.2152


In [ ]:
predictions = model.predict(X_test)

14150/14150 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step


In [ ]:
mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)
print(f"Mean Squared Error (MSE): {mse:.4f}, Mean Absolute Error (MAE): {mae:.4f}, R-squared (R²): {r2:.4f}")

Mean Squared Error (MSE): 735.2148, Mean Absolute Error (MAE): 6.2246, R-squared (R²): 0.9596
